![Two data scientists working on a dashboard.](hr-image-small.png)

A common problem when creating models to generate business value from data is that the datasets can be so large that it can take days for the model to generate predictions. Ensuring that your dataset is stored as efficiently as possible is crucial for allowing these models to run on a more reasonable timescale without having to reduce the size of the dataset.

You've been hired by a major online data science training provider called *Training Data Ltd.* to clean up one of their largest customer datasets. This dataset will eventually be used to predict whether their students are looking for a new job or not, information that they will then use to direct them to prospective recruiters.

You've been given access to `customer_train.csv`, which is a subset of their entire customer dataset, so you can create a proof-of-concept of a much more efficient storage solution. The dataset contains anonymized student information, and whether they were looking for a new job or not during training:

| Column                   | Description                                                                      |
|------------------------- |--------------------------------------------------------------------------------- |
| `student_id`             | A unique ID for each student.                                                    |
| `city`                   | A code for the city the student lives in.                                        |
| `city_development_index` | A scaled development index for the city.                                         |
| `gender`                 | The student's gender.                                                            |
| `relevant_experience`    | An indicator of the student's work relevant experience.                          |
| `enrolled_university`    | The type of university course enrolled in (if any).                              |
| `education_level`        | The student's education level.                                                   |
| `major_discipline`       | The educational discipline of the student.                                       |
| `experience`             | The student's total work experience (in years).                                  |
| `company_size`           | The number of employees at the student's current employer.                       |
| `company_type`           | The type of company employing the student.                                       |
| `last_new_job`           | The number of years between the student's current and previous jobs.             |
| `training_hours`         | The number of hours of training completed.                                       |
| `job_change`             | An indicator of whether the student is looking for a new job (`1`) or not (`0`). |

In [1]:
# Import necessary libraries
import pandas as pd

# Load the dataset
ds_jobs = pd.read_csv("customer_train.csv")

# View the dataset
ds_jobs.head()

,student_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,job_change
0,8949,city_103,0.920,Male,Has relevant experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1.0
1,29725,city_40,0.776,Male,No relevant experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0.0
2,11561,city_21,0.624,NaN,No relevant experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0.0
3,33241,city_115,0.789,NaN,No relevant experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1.0
4,666,city_162,0.767,Male,Has relevant experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0.0


In [2]:
# Create a copy of ds_jobs for transforming
ds_jobs_transformed = ds_jobs.copy()

# Start coding here. Use as many cells as you like!

In [3]:
ds_jobs_transformed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   student_id              19158 non-null  int64  
 1   city                    19158 non-null  object 
 2   city_development_index  19158 non-null  float64
 3   gender                  14650 non-null  object 
 4   relevant_experience     19158 non-null  object 
 5   enrolled_university     18772 non-null  object 
 6   education_level         18698 non-null  object 
 7   major_discipline        16345 non-null  object 
 8   experience              19093 non-null  object 
 9   company_size            13220 non-null  object 
 10  company_type            13018 non-null  object 
 11  last_new_job            18735 non-null  object 
 12  training_hours          19158 non-null  int64  
 13  job_change              19158 non-null  float64
dtypes: float64(2), int64(2), object(10)
me

## Converting integers, floats, and unordered categories

In [4]:
# Transform columns with integer values from int64 to int32
ds_jobs_transformed['student_id'] = ds_jobs_transformed['student_id'].astype('int32')
ds_jobs_transformed['training_hours'] = ds_jobs_transformed['training_hours'].astype('int32')

In [5]:
# Transform column with float values from float64 to float16
ds_jobs_transformed['city_development_index'] = ds_jobs_transformed['city_development_index'].astype('float16')
# ds_jobs_transformed['job_change'] = ds_jobs_transformed['job_change'].astype('float16') -> this one in fact has only two unique values


In [6]:
ds_jobs_transformed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   student_id              19158 non-null  int32  
 1   city                    19158 non-null  object 
 2   city_development_index  19158 non-null  float16
 3   gender                  14650 non-null  object 
 4   relevant_experience     19158 non-null  object 
 5   enrolled_university     18772 non-null  object 
 6   education_level         18698 non-null  object 
 7   major_discipline        16345 non-null  object 
 8   experience              19093 non-null  object 
 9   company_size            13220 non-null  object 
 10  company_type            13018 non-null  object 
 11  last_new_job            18735 non-null  object 
 12  training_hours          19158 non-null  int32  
 13  job_change              19158 non-null  float64
dtypes: float16(1), float64(1), int32(2), o

In [7]:
ds_jobs_transformed['relevant_experience'].value_counts()

relevant_experience
Has relevant experience    13792
No relevant experience      5366
Name: count, dtype: int64

In [8]:
ds_jobs_transformed['job_change'].value_counts()

job_change
0.0    14381
1.0     4777
Name: count, dtype: int64

In [9]:
# Let's convert the two factors column into bool
ds_jobs_transformed['job_change'] = ds_jobs_transformed['job_change'].astype('bool')
ds_jobs_transformed['relevant_experience'] = ds_jobs_transformed['relevant_experience'] == 'Has relevant experience'
    

In [10]:
# Let us check the data types of the transformed data
ds_jobs_transformed.dtypes

student_id                  int32
city                       object
city_development_index    float16
gender                     object
relevant_experience          bool
enrolled_university        object
education_level            object
major_discipline           object
experience                 object
company_size               object
company_type               object
last_new_job               object
training_hours              int32
job_change                   bool
dtype: object

In [11]:
ds_jobs_transformed['job_change'].value_counts()

job_change
False    14381
True      4777
Name: count, dtype: int64

In [12]:
ds_jobs_transformed['relevant_experience'].value_counts()

relevant_experience
True     13792
False     5366
Name: count, dtype: int64

In [13]:
# Let's unordered categoriesconvert object columns into categorical type
for col in ds_jobs_transformed.select_dtypes(include=['object']).columns:
    print(f"{col}: {ds_jobs_transformed[col].nunique()}")

city: 123
gender: 3
enrolled_university: 3
education_level: 5
major_discipline: 6
experience: 22
company_size: 8
company_type: 6
last_new_job: 6


In [14]:
for col in ds_jobs_transformed.select_dtypes(include=['object']).columns[1:]:
    print(ds_jobs_transformed[col].value_counts())

gender
Male      13221
Female     1238
Other       191
Name: count, dtype: int64
enrolled_university
no_enrollment       13817
Full time course     3757
Part time course     1198
Name: count, dtype: int64
education_level
Graduate          11598
Masters            4361
High School        2017
Phd                 414
Primary School      308
Name: count, dtype: int64
major_discipline
STEM               14492
Humanities           669
Other                381
Business Degree      327
Arts                 253
No Major             223
Name: count, dtype: int64
experience
>20    3286
5      1430
4      1403
3      1354
6      1216
2      1127
7      1028
10      985
9       980
8       802
15      686
11      664
14      586
1       549
<1      522
16      508
12      494
13      399
17      342
19      304
18      280
20      148
Name: count, dtype: int64
company_size
50-99        3083
100-499      2571
10000+       2019
10-49        1471
1000-4999    1328
<10          1308
500-999       877


From the outputs we can conclude that the unoreder categories would be:
'city', 'gender', 'enrolled_university', 'major_discipline', 'company_type'


In [15]:
# converting objects to nominal categories 
nominal_categories = [
    'city',
    'gender',
    'enrolled_university',
    'major_discipline',
    'company_type'
    ]

for col in nominal_categories:
    ds_jobs_transformed[col] = ds_jobs_transformed[col].astype('category')

In [16]:
ds_jobs_transformed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   student_id              19158 non-null  int32   
 1   city                    19158 non-null  category
 2   city_development_index  19158 non-null  float16 
 3   gender                  14650 non-null  category
 4   relevant_experience     19158 non-null  bool    
 5   enrolled_university     18772 non-null  category
 6   education_level         18698 non-null  object  
 7   major_discipline        16345 non-null  category
 8   experience              19093 non-null  object  
 9   company_size            13220 non-null  object  
 10  company_type            13018 non-null  category
 11  last_new_job            18735 non-null  object  
 12  training_hours          19158 non-null  int32   
 13  job_change              19158 non-null  bool    
dtypes: bool(2), category(5

## Converting ordered categories

In [17]:
ordered_categories = [col for col in ds_jobs_transformed.select_dtypes(include=['object']).columns]
ordered_categories

['education_level', 'experience', 'company_size', 'last_new_job']

In [18]:
# Converting educational_level column
ds_jobs['education_level'].value_counts()

education_level
Graduate          11598
Masters            4361
High School        2017
Phd                 414
Primary School      308
Name: count, dtype: int64

In [19]:
education_level = [
    'Primary School',
    'High School',
    'Graduate',
    'Masters',
    'Phd'
]

ds_jobs_transformed['education_level'] = pd.Categorical(
    ds_jobs_transformed['education_level'],
    categories=education_level,
    ordered=True
)

In [20]:
ds_jobs_transformed['education_level'].dtype

CategoricalDtype(categories=['Primary School', 'High School', 'Graduate', 'Masters',
                  'Phd'],
, ordered=True, categories_dtype=object)

In [21]:
# Converting 'experience' column
ds_jobs_transformed['experience'].value_counts()

experience
>20    3286
5      1430
4      1403
3      1354
6      1216
2      1127
7      1028
10      985
9       980
8       802
15      686
11      664
14      586
1       549
<1      522
16      508
12      494
13      399
17      342
19      304
18      280
20      148
Name: count, dtype: int64

In [34]:
# Create numeric version for ordering logic, but keep as ordered categorical
exp_categories = ['<1'] + [str(i) for i in range(1, 21)] + ['>20']
print(exp_categories)
ds_jobs_transformed['experience'] = pd.Categorical(
    ds_jobs_transformed['experience'],
    categories=exp_categories,
    ordered=True
)
ds_jobs_transformed['experience']

['<1', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '>20']


0        >20
1         15
2          5
3         <1
4        >20
        ... 
19153     14
19154     14
19155    >20
19156     <1
19157      2
Name: experience, Length: 19158, dtype: category
Categories (22, object): ['<1' < '1' < '2' < '3' ... '18' < '19' < '20' < '>20']

In [23]:
# Converting company_size column into ordered category
list(ds_jobs_transformed['company_size'].value_counts().index)

['50-99',
 '100-499',
 '10000+',
 '10-49',
 '1000-4999',
 '<10',
 '500-999',
 '5000-9999']

In [24]:
order_company_size = [
    '<10',
    '10-49',
    '50-99',
    '100-499',
    '500-999',
    '1000-4999',
    '5000-9999',
    '10000+'
]

ds_jobs_transformed['company_size'] = pd.Categorical(
    ds_jobs_transformed['company_size'],
    categories=order_company_size,
    ordered=True
    )

In [25]:
# Convert column last_new_job to ordered categorical
ds_jobs_transformed['last_new_job'].value_counts()

last_new_job
1        8040
>4       3290
2        2900
never    2452
4        1029
3        1024
Name: count, dtype: int64

In [26]:
list(ds_jobs_transformed['last_new_job'].value_counts().index)

['1', '>4', '2', 'never', '4', '3']

In [27]:
order_last_new_job = ['never', '1', '2', '3', '4', '>4']
ds_jobs_transformed['last_new_job'] = pd.Categorical(
    ds_jobs_transformed['last_new_job'],
    categories=order_last_new_job,
    ordered=True
    )

## Memory Usage before and after preprocessing

In [28]:
 ds_jobs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   student_id              19158 non-null  int64  
 1   city                    19158 non-null  object 
 2   city_development_index  19158 non-null  float64
 3   gender                  14650 non-null  object 
 4   relevant_experience     19158 non-null  object 
 5   enrolled_university     18772 non-null  object 
 6   education_level         18698 non-null  object 
 7   major_discipline        16345 non-null  object 
 8   experience              19093 non-null  object 
 9   company_size            13220 non-null  object 
 10  company_type            13018 non-null  object 
 11  last_new_job            18735 non-null  object 
 12  training_hours          19158 non-null  int64  
 13  job_change              19158 non-null  float64
dtypes: float64(2), int64(2), object(10)
me

In [29]:
ds_jobs_transformed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   student_id              19158 non-null  int32   
 1   city                    19158 non-null  category
 2   city_development_index  19158 non-null  float16 
 3   gender                  14650 non-null  category
 4   relevant_experience     19158 non-null  bool    
 5   enrolled_university     18772 non-null  category
 6   education_level         18698 non-null  category
 7   major_discipline        16345 non-null  category
 8   experience              19093 non-null  category
 9   company_size            13220 non-null  category
 10  company_type            13018 non-null  category
 11  last_new_job            18735 non-null  category
 12  training_hours          19158 non-null  int32   
 13  job_change              19158 non-null  bool    
dtypes: bool(2), category(9

## Filtering on ordered categorical columns

In [38]:
# Filter the DataFrame to only contain students with 10 or more years of experience at companies with at least 1000 employees.
ds_jobs_filtered = ds_jobs_transformed[
    (ds_jobs_transformed['experience'] >= '10') & 
    (ds_jobs_transformed['company_size'] >= '1000-4999')
]
ds_jobs_filtered

/home/tom/training/projects/ds_jobs/.venv/lib/python3.14/site-packages/pandas/io/formats/format.py:1458: RuntimeWarning: overflow encountered in cast
  has_large_values = (abs_vals > 1e6).any()


,student_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,job_change
9,699,city_103,0.919922,NaN,True,no_enrollment,Graduate,STEM,17,10000+,Pvt Ltd,>4,123,False
12,25619,city_61,0.913086,Male,True,no_enrollment,Graduate,STEM,>20,1000-4999,Pvt Ltd,3,23,False
31,22293,city_103,0.919922,Male,True,Part time course,Graduate,STEM,19,5000-9999,Pvt Ltd,>4,141,False
34,26494,city_16,0.910156,Male,True,no_enrollment,Graduate,Business Degree,12,5000-9999,Pvt Ltd,3,145,False
40,2547,city_114,0.925781,Female,True,Full time course,Masters,STEM,16,1000-4999,Public Sector,2,14,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19097,25447,city_103,0.919922,Male,True,no_enrollment,Graduate,STEM,>20,1000-4999,Pvt Ltd,>4,57,False
19101,6803,city_16,0.910156,Male,True,no_enrollment,High School,NaN,10,10000+,Pvt Ltd,1,89,False
19103,32932,city_10,0.895020,Male,True,Part time course,Masters,Other,>20,1000-4999,Pvt Ltd,>4,18,False
19128,3365,city_16,0.910156,NaN,True,no_enrollment,Graduate,Humanities,>20,1000-4999,Pvt Ltd,>4,23,False


In [ ]:
# Correction for enrolled_university

order_enrolled_university = ['no_enrollment', 'Part time course', 'Full time course']

ds_jobs_transformed['enrolled_university'] = pd.Categorical(
    ds_jobs_transformed['enrolled_university'],
    categories=order_enrolled_university,
    ordered=True
)
ds_jobs_transformed['enrolled_university'].dtype


CategoricalDtype(categories=['no_enrollment', 'Part time course', 'Full time course'], ordered=True, categories_dtype=object)

In [39]:
# corrected last - ds_jobs_trasformed needs to contain only filtered students
ds_jobs_transformed = ds_jobs_filtered